##### Copyright 2019 Google LLC

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

On data 
*Terrorists are linked to each other if they contact each other, use the same facility, are members of the same family, or belong to the same terrorist organization.*

# Graph regularization for document classification using natural graphs

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_mlp_cora"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/neural-structured-learning/blob/master/g3doc/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/neural-structured-learning/blob/master/g3doc/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

Graph regularization is a specific technique under the broader paradigm of
Neural Graph Learning
([Bui et al., 2018](https://ai.google/research/pubs/pub46568.pdf)). The core
idea is to train neural network models with a graph-regularized objective,
harnessing both labeled and unlabeled data.

In this tutorial, we will explore the use of graph regularization to classify
documents that form a natural (organic) graph.

The general recipe for creating a graph-regularized model using the Neural
Structured Learning (NSL) framework is as follows:

1.  Generate training data from the input graph and sample features. Nodes in
    the graph correspond to samples and edges in the graph correspond to
    similarity between pairs of samples. The resulting training data will
    contain neighbor features in addition to the original node features.
2.  Create a neural network as a base model using the `Keras` sequential,
    functional, or subclass API.
3.  Wrap the base model with the **`GraphRegularization`** wrapper class, which
    is provided by the NSL framework, to create a new graph `Keras` model. This
    new model will include a graph regularization loss as the regularization
    term in its training objective.
4.  Train and evaluate the graph `Keras` model.

## Setup



1.  Select TensorFlow 2.x to create an interactive development environment with eager execution.
2.  Install the Neural Structured Learning package.

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  !pip install tensorflow-gpu>=2.0.0

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu


In [2]:
!pip install --quiet neural-structured-learning

## Dependencies and imports

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import neural_structured_learning as nsl

import tensorflow as tf

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.0.0-rc1
Eager mode:  True
GPU is NOT AVAILABLE


## Cora dataset

The [Cora dataset](https://linqs.soe.ucsc.edu/data) is a citation graph where
nodes represent machine learning papers and edges represent citations between
pairs of papers. The task involved is document classification where the goal is
to categorize each paper into one of 7 categories. In other words, this is a
multi-class classification problem with 7 classes.

### Graph

The original graph is directed. However, for the purpose of this example, we
consider the undirected version of this graph. So, if paper A cites paper B, we
also consider paper B to have cited A. Although this is not necessarily true, in
this example, we consider citations as a proxy for similarity, which is usually
a commutative property.

### Features

Each paper in the input effectively contains 2 features:

1.  **Words**: A dense, multi-hot bag-of-words representation of the text in the
    paper. The vocabulary for the Cora dataset contains 1433 unique words. So,
    the length of this feature is 1433, and the value at position 'i' is 0/1
    indicating whether word 'i' in the vocabulary exists in the given paper or
    not.

2.  **Label**: A single integer representing the class ID (category) of the paper.

### Download the Cora dataset

In [7]:
!wget --quiet -P /tmp https://linqs-data.soe.ucsc.edu/public/lbc/TerrorAttack.tgz
# !wget --quiet -P /tmp https://github.com/mrweissguy/nsl/tree/master/data/TerroristRel
    
    
    
# !tar -C /tmp -xvzf /tmp/cora.tgz
!tar -C /tmp -xvzf /tmp/TerrorAttack.tgz 

x TerrorAttack/
x TerrorAttack/terrorist_attack.nodes
x TerrorAttack/terrorist_attack_loc_org.edges
x TerrorAttack/README
x TerrorAttack/terrorist_attack_loc.edges
x TerrorAttack/terrorist_attack.labels


### Convert the Cora data to the NSL format

In order to preprocess the Cora dataset and convert it to the format required by
Neural Structured Learning, we will run the **'preprocess_cora_dataset.py'**
script, which is included in the NSL github repository. This script does the
following:

1.  Generate neighbor features using the original node features and the graph.
2.  Generate train and test data splits containing `tf.train.Example` instances.
3.  Persist the resulting train and test data in the `TFRecord` format.

In [11]:
# !wget https://raw.githubusercontent.com/tensorflow/neural-structured-learning/master/neural_structured_learning/examples/preprocess/cora/preprocess_cora_dataset.py

# !python preprocess_cora_dataset.py \
# --input_cora_content=/tmp/cora/cora.content \
# --input_cora_graph=/tmp/cora/cora.cites \
# --max_nbrs=5 \
# --output_train_data=/tmp/cora/train_merged_examples.tfr \
# --output_test_data=/tmp/cora/test_examples.tfr

# !wget https://raw.githubusercontent.com/mrweissguy/nsl/master/preprocessing_terror_dataset.py?token=AKMVMPGQOLZCWR6HB2NCZQK5XGN6U
# https://raw.githubusercontent.com/mrweissguy/nsl/master/preprocessing_terror_dataset.py?token=AKMVMPEPFXMPBD5SDWZSSOS5XF6NO
!wget https://raw.githubusercontent.com/mrweissguy/nsl/master/dataset_test_2/preprocessing_terror_dataset.py?token=AKMVMPA3BIMHYWRBUIAZLOK5YEXHC
    
  
!python preprocessing_terror_dataset.py \
--input_cora_content=/tmp/TerrorAttack/terrorist_attack.nodes \
--input_cora_graph=/tmp/TerrorAttack/terrorist_attack_loc.edges \
--max_nbrs=5 \
--output_train_data=/tmp/TerrorAttack/train_merged_examples.tfr \
--output_test_data=/tmp/TerrorAttack/test_examples.tfr

--2019-11-05 09:11:20--  https://raw.githubusercontent.com/mrweissguy/nsl/master/dataset_test_2/preprocessing_terror_dataset.py?token=AKMVMPA3BIMHYWRBUIAZLOK5YEXHC
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.40.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.40.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12747 (12K) [text/plain]
Saving to: ‘preprocessing_terror_dataset.py?token=AKMVMPA3BIMHYWRBUIAZLOK5YEXHC.1’

preprocessing_terro 100%[===================>]  12.45K  --.-KB/s    in 0.04s   

2019-11-05 09:11:21 (296 KB/s) - ‘preprocessing_terror_dataset.py?token=AKMVMPA3BIMHYWRBUIAZLOK5YEXHC.1’ saved [12747/12747]

preprocessing_terror_dataset.py:144: DeprecationWarning: 'U' mode is deprecated
  with open(in_file, 'rU') as cora_content:
Reading graph file: /tmp/TerrorAttack/terrorist_attack_loc.edges...
Traceback (most recent call last):
  File "preprocessing_terror_dataset.py", line 305, in

## Global variables

The file paths to the train and test data are based on the command line flag
values used to invoke the **'preprocess_cora_dataset.py'** script above.

In [8]:
# ### Experiment dataset
# TRAIN_DATA_PATH = '/tmp/cora/train_merged_examples.tfr'
# TEST_DATA_PATH = '/tmp/cora/test_examples.tfr'

# ### Constants used to identify neighbor features in the input.
# NBR_FEATURE_PREFIX = 'NL_nbr_'
# NBR_WEIGHT_SUFFIX = '_weight'

### Experiment dataset
TRAIN_DATA_PATH = '/tmp/TerroristRel/train_merged_examples.tfr'
TEST_DATA_PATH = '/tmp/TerroristRel/test_examples.tfr'

### Constants used to identify neighbor features in the input.
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'



## Hyperparameters

We will use an instance of `HParams` to inclue various hyperparameters and
constants used for training and evaluation. We briefly describe each of them
below:

-   **num_classes**: There are a total 7 different classes

-   **max_seq_length**: This is the size of the vocabulary and all instances in
    the input have a dense multi-hot, bag-of-words representation. In other
    words, a value of 1 for a word indicates that the word is present in the
    input and a value of 0 indicates that it is not.

-   **distance_type**: This is the distance metric used to regularize the sample
    with its neighbors.

-   **graph_regularization_multiplier**: This controls the relative weight of
    the graph regularization term in the overall loss function.

-   **num_neighbors**: The number of neighbors used for graph regularization.
    This value has to be less than or equal to the `max_nbrs` command-line
    argument used above when running `preprocess_cora_dataset.py`.

-   **num_fc_units**: The number of fully connected layers in our neural
    network.

-   **train_epochs**: The number of training epochs.

-   **batch_size**: Batch size used for training and evaluation.

-   **dropout_rate**: Controls the rate of dropout following each fully
    connected layer

-   **eval_steps**: The number of batches to process before deeming evaluation
    is complete. If set to `None`, all instances in the test set are evaluated.

In [9]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### dataset parameters
    # self.num_classes = 7
    self.num_classes = 2
    self.max_seq_length = 1224 # distinct features
    ### neural graph learning parameters
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 1
    ### model architecture
    self.num_fc_units = [50, 50]
    ### training parameters
    self.train_epochs = 100
    self.batch_size = 128
    self.dropout_rate = 0.5
    ### eval parameters
    self.eval_steps = None  # All instances in the test set are evaluated.

HPARAMS = HParams()

## Load train and test data

As described earlier in this notebook, the input training and test data have
been created by the **'preprocess_cora_dataset.py'**. We will load them into two
`tf.data.Dataset` objects -- one for train and one for test.

In the input layer of our model, we will extract not just the 'words' and the
'label' features from each sample, but also corresponding neighbor features
based on the `hparams.num_neighbors`. Instances with fewer neighbors than
`hparams.num_neighbors` will be assigned dummy values for those non-existent
neighbor features.

In [10]:
def parse_example(example_proto):
  """Extracts relevant fields from the `example_proto`.

  Args:
    example_proto: An instance of `tf.train.Example`.

  Returns:
    A pair whose first value is a dictionary containing relevant features
    and whose second value contains the ground truth labels.
  """
  # The 'words' feature is a multi-hot, bag-of-words representation of the
  # original raw text. A default value is required for examples that don't
  # have the feature.
  feature_spec = {
      'words':
          tf.io.FixedLenFeature([HPARAMS.max_seq_length],
                                tf.int64,
                                default_value=tf.constant(
                                    0,
                                    dtype=tf.int64,
                                    shape=[HPARAMS.max_seq_length])),
      'label':
          tf.io.FixedLenFeature((), tf.int64, default_value=-1),
  }
  # We also extract corresponding neighbor features in a similar manner to
  # the features above.
  for i in range(HPARAMS.num_neighbors):
    nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
    nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i, NBR_WEIGHT_SUFFIX)
    feature_spec[nbr_feature_key] = tf.io.FixedLenFeature(
        [HPARAMS.max_seq_length],
        tf.int64,
        default_value=tf.constant(
            0, dtype=tf.int64, shape=[HPARAMS.max_seq_length]))

    # We assign a default value of 0.0 for the neighbor weight so that
    # graph regularization is done on samples based on their exact number
    # of neighbors. In other words, non-existent neighbors are discounted.
    feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
        [1], tf.float32, default_value=tf.constant([0.0]))

  features = tf.io.parse_single_example(example_proto, feature_spec)

  labels = features.pop('label')
  return features, labels


def make_dataset(file_path, training=False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """
  dataset = tf.data.TFRecordDataset([file_path])
  if training:
    dataset = dataset.shuffle(10000)
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  return dataset


train_dataset = make_dataset(TRAIN_DATA_PATH, training=True)
test_dataset = make_dataset(TEST_DATA_PATH)

Let's peek into the train dataset to look at its contents.

In [11]:
for feature_batch, label_batch in train_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, 0, 'words')
  nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, 0, NBR_WEIGHT_SUFFIX)
  print('Batch of neighbor inputs:', feature_batch[nbr_feature_key])
  print('Batch of neighbor weights:',
        tf.reshape(feature_batch[nbr_weight_key], [-1]))
  print('Batch of labels:', label_batch)

Feature list: ['NL_nbr_0_weight', 'NL_nbr_0_words', 'words']
Batch of inputs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1224), dtype=int64)
Batch of neighbor inputs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1224), dtype=int64)
Batch of neighbor weights: tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.], shape=(128,), dtype=float32)
Batch of labels: tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0
 1 1 1

Let's peek into the test dataset to look at its contents.

In [12]:
for feature_batch, label_batch in test_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, 0, 'words')
  nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, 0, NBR_WEIGHT_SUFFIX)
  print('Batch of neighbor inputs:', feature_batch[nbr_feature_key])
  print('Batch of neighbor weights:',
        tf.reshape(feature_batch[nbr_weight_key], [-1]))
  print('Batch of labels:', label_batch)

Feature list: ['NL_nbr_0_weight', 'NL_nbr_0_words', 'words']
Batch of inputs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1224), dtype=int64)
Batch of neighbor inputs: tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(128, 1224), dtype=int64)
Batch of neighbor weights: tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(128,), dtype=float32)
Batch of labels: tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1

## Model definition

In order to demonstrate the use of graph regularization, we build a base model
for this problem first. We will use a simple feed-forward neural network with 2
hidden layers and dropout in between. We illustrate the creation of the base
model using all model types supported by the `tf.Keras` framework -- sequential,
functional, and subclass.

### Sequential base model

In [13]:
def make_mlp_sequential_model(hparams):
  """Creates a sequential multi-layer perceptron model."""
  model = tf.keras.Sequential()
  model.add(
      tf.keras.layers.InputLayer(
          input_shape=(hparams.max_seq_length,), name='words'))
  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  model.add(
      tf.keras.layers.Lambda(lambda x: tf.keras.backend.cast(x, tf.float32)))
  for num_units in hparams.num_fc_units:
    model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    # For sequential models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    model.add(tf.keras.layers.Dropout(hparams.dropout_rate))
  model.add(tf.keras.layers.Dense(hparams.num_classes, activation='softmax'))
  return model

### Functional base model

In [14]:
def make_mlp_functional_model(hparams):
  """Creates a functional API-based multi-layer perceptron model."""
  inputs = tf.keras.Input(
      shape=(hparams.max_seq_length,), dtype='int64', name='words')

  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  cur_layer = tf.keras.layers.Lambda(
      lambda x: tf.keras.backend.cast(x, tf.float32))(
          inputs)

  for num_units in hparams.num_fc_units:
    cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
    # For functional models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    cur_layer = tf.keras.layers.Dropout(hparams.dropout_rate)(cur_layer)

  outputs = tf.keras.layers.Dense(
      hparams.num_classes, activation='softmax')(
          cur_layer)

  model = tf.keras.Model(inputs, outputs=outputs)
  return model

### Subclass base model

In [15]:
def make_mlp_subclass_model(hparams):
  """Creates a multi-layer perceptron subclass model in Keras."""

  class MLP(tf.keras.Model):
    """Subclass model defining a multi-layer perceptron."""

    def __init__(self):
      super(MLP, self).__init__()
      # Input is already one-hot encoded in the integer format. We create a
      # layer to cast it to floating point format here.
      self.cast_to_float_layer = tf.keras.layers.Lambda(
          lambda x: tf.keras.backend.cast(x, tf.float32))
      self.dense_layers = [
          tf.keras.layers.Dense(num_units, activation='relu')
          for num_units in hparams.num_fc_units
      ]
      self.dropout_layer = tf.keras.layers.Dropout(hparams.dropout_rate)
      self.output_layer = tf.keras.layers.Dense(
          hparams.num_classes, activation='softmax')

    def call(self, inputs, training=False):
      cur_layer = self.cast_to_float_layer(inputs['words'])
      for dense_layer in self.dense_layers:
        cur_layer = dense_layer(cur_layer)
        cur_layer = self.dropout_layer(cur_layer, training=training)

      outputs = self.output_layer(cur_layer)

      return outputs

  return MLP()

## Create base model(s)

In [16]:
# Create a base MLP model using the functional API.
# Alternatively, you can also create a sequential or subclass base model using
# the make_mlp_sequential_model() or make_mlp_subclass_model() functions
# respectively, defined above. Note that if a subclass model is used, its
# summary cannot be generated until it is built.
base_model_tag, base_model = 'FUNCTIONAL', make_mlp_functional_model(HPARAMS)
base_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
words (InputLayer)           [(None, 1224)]            0         
_________________________________________________________________
lambda (Lambda)              (None, 1224)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                61250     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102   

## Train base (Multilayer perceptron) MLP model

In [17]:
# Compile and train the base MLP model
base_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
base_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

Epoch 1/100
6/6 [==============================] - 1s 93ms/step - loss: 0.6286 - accuracy: 0.7859
Epoch 2/100
6/6 [==============================] - 0s 15ms/step - loss: 0.5799 - accuracy: 0.8174
Epoch 3/100
6/6 [==============================] - 0s 13ms/step - loss: 0.5328 - accuracy: 0.8219
Epoch 4/100
6/6 [==============================] - 0s 19ms/step - loss: 0.4964 - accuracy: 0.8219
Epoch 5/100
6/6 [==============================] - 0s 25ms/step - loss: 0.4691 - accuracy: 0.8234
Epoch 6/100
6/6 [==============================] - 0s 23ms/step - loss: 0.4394 - accuracy: 0.8234
Epoch 7/100
6/6 [==============================] - 0s 22ms/step - loss: 0.4157 - accuracy: 0.8219
Epoch 8/100
6/6 [==============================] - 0s 21ms/step - loss: 0.4082 - accuracy: 0.8219
Epoch 9/100
6/6 [==============================] - 0s 22ms/step - loss: 0.3827 - accuracy: 0.8263
Epoch 10/100
6/6 [==============================] - 0s 22ms/step - loss: 0.3773 - accuracy: 0.8219
Epoch 11/100
6/6 [=

## Evaluate base MLP model

In [18]:
# Helper function to print evaluation metrics.
def print_metrics(model_desc, eval_metrics):
  """Prints evaluation metrics.

  Args:
    model_desc: A description of the model.
    eval_metrics: A dictionary mapping metric names to corresponding values. It
      must contain the loss and accuracy metrics.
  """
  print('\n')
  print('Eval accuracy for ', model_desc, ': ', eval_metrics['accuracy'])
  print('Eval loss for ', model_desc, ': ', eval_metrics['loss'])
  if 'graph_loss' in eval_metrics:
    print('Eval graph loss for ', model_desc, ': ', eval_metrics['graph_loss'])

In [19]:
eval_results = dict(
    zip(base_model.metrics_names,
        base_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('Base MLP model', eval_results)

2/2 [==============================] - 0s 51ms/step - loss: 0.5879 - accuracy: 0.8634


Eval accuracy for  Base MLP model :  0.863388
Eval loss for  Base MLP model :  0.5878893360495567


## Train MLP model with graph regularization

Incorporating graph regularization into the loss term of an existing
`tf.Keras.Model` requires just a few lines of code. The base model is wrapped to
create a new `tf.Keras` subclass model, whose loss includes graph
regularization.

To assess the incremental benefit of graph regularization, we will create a new
base model instance. This is because `base_model` has already been trained for a
few iterations, and reusing this trained model to create a graph-regularized
model will not be a fair comparison for `base_model`.

In [20]:
# Build a new base MLP model.
base_reg_model_tag, base_reg_model = 'FUNCTIONAL', make_mlp_functional_model(
    HPARAMS)

In [21]:
# Wrap the base MLP model with graph regularization.
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=HPARAMS.num_neighbors,
    multiplier=HPARAMS.graph_regularization_multiplier,
    distance_type=HPARAMS.distance_type,
    sum_over_axis=-1)

graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
graph_reg_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100


/Users/johanweisshansen/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/Users/johanweisshansen/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


6/6 [==============================] - 2s 319ms/step - loss: 0.6525 - accuracy: 0.6377 - graph_loss: 0.0199
Epoch 2/100
6/6 [==============================] - 0s 28ms/step - loss: 0.6235 - accuracy: 0.8024 - graph_loss: 0.0228
Epoch 3/100
6/6 [==============================] - 0s 26ms/step - loss: 0.5831 - accuracy: 0.8189 - graph_loss: 0.0278
Epoch 4/100
6/6 [==============================] - 0s 24ms/step - loss: 0.5321 - accuracy: 0.8249 - graph_loss: 0.0266
Epoch 5/100
6/6 [==============================] - 0s 24ms/step - loss: 0.4975 - accuracy: 0.8234 - graph_loss: 0.0317
Epoch 6/100
6/6 [==============================] - 0s 28ms/step - loss: 0.4665 - accuracy: 0.8293 - graph_loss: 0.0325
Epoch 7/100
6/6 [==============================] - 0s 27ms/step - loss: 0.4432 - accuracy: 0.8278 - graph_loss: 0.0347
Epoch 8/100
6/6 [==============================] - 0s 27ms/step - loss: 0.3978 - accuracy: 0.8293 - graph_loss: 0.0363
Epoch 9/100
6/6 [==============================] - 0s 26ms/

## Evaluate MLP model with graph regularization

In [22]:
eval_results = dict(
    zip(graph_reg_model.metrics_names,
        graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('MLP + graph regularization', eval_results)

2/2 [==============================] - 0s 159ms/step - loss: 0.4977 - accuracy: 0.9071 - graph_loss: 0.0000e+00


Eval accuracy for  MLP + graph regularization :  0.90710384
Eval loss for  MLP + graph regularization :  0.4976902939379215
Eval graph loss for  MLP + graph regularization :  0.0


The graph-regularized model's accuracy is about 2-3% higher than that of the
base model (`base_model`).

## Conclusion

We have demonstrated the use of graph regularization for document classification
on a natural citation graph (Cora) using the Neural Structured Learning (NSL)
framework. Our [advanced tutorial](graph_keras_lstm_imdb.ipynb) involves
synthesizing graphs based on sample embeddings before training a neural network
with graph regularization. This approach is useful if the input does not contain
an explicit graph.

We encourage users to experiment further by varying the amount of supervision as
well as trying different neural architectures for graph regularization.